<h1 align="center"><font size="7" face="arial" color="#DC5A29">First steps using TVB</font></h1>

<h2 align="center"><font size="5" face="arial">EITN Spring School in Computational Neuroscience, 4-13 March 2020</font></h2>

<h1><font size="6" face="arial" color="#609BC4">Context</font></h1>

***

<p><div style="text-align: justify"><font size="4.5" face="time roman">A current topic in system neuroscience literature is the presence of brain activity in the absence of a task condition. These task-negative, spontaneous fluctuations occur in the so-called <b>rest state</b>, and a recurring theme of these fluctuations is that they have a network structure. Because TVB uses the structural connectivity of the brain as the backbone for simulating spontaneous activity, resting-state activity and its network structure is a prime candidate for modeling in TVB.</font></div></p>

<h1><font size="6" face="arial" color="#609BC4">Objectives</font></h1>

***

<p><div style="text-align: justify"><font size="4.5" face="time roman">In this mini-project, we will:
<br>
<ul>
    <li>Build a brain network model using subject-specific structural connectivity,</li> 
    <li>Simulate resting-state activity,</li>
    <li>Characterize the resting-state activity by calculating the functional connectivity (FC),</li>
    <li>Extract the resting-state networks (RSNs).</li>
</ul></font></div></p>

<h1><font size="6" face="arial" color="#609BC4">How to do it with TVB?</font></h1> 

***

<p><div style="text-align: justify"><font size="4.5" face="time roman">In the first part of this tutorial, we presents the basic anatomy of a region simulation using The Virtual Brain's scripting interface.</font></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman">The first thing we want to do is to import the modules we will need for a simulation.</font></div></p>

In [ ]:
import time as tm

import numpy as np
import matplotlib.pyplot as plt        
import matplotlib.gridspec as gridspec 

# Import a bunch of stuff to ease command line usage
from tvb.simulator.lab import *
from tvb.simulator.plot.phase_plane_interactive import PhasePlaneInteractive

%matplotlib notebook

<h1><font size="6" face="arial" color="black">1. Setting up the simulation</font></h1>

<p><div style="text-align: justify"><font size="4.5" face="time roman">A basic simulation of TVB consists of <b>5 main components</b>. Each of these components is an object within TVB:</font>

- ## <font size="5" face="arial" color="black"> Connectivity</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">We start by loading and visualizing the structural connectivity matrix that represents the set of all existing connections between brain areas. Having loaded the default dataset, we can then alter the speed of signal propagation through the network:</font></div></p>

In [ ]:
# Import the anatomical structural connectivity.
conn = connectivity.Connectivity().from_file()      
nregions = len(conn.region_labels)               # Number of regions
conn.speed = np.array(np.inf)                    # Set the conduction speed
conn.configure()

sc = conn.weights

In [ ]:
# Visualization
fig = plt.figure(figsize=(8, 4))
fig.suptitle('TVB SC', fontsize=20)

# Weights
plt.subplot(121)
plt.imshow(conn.weights, interpolation='nearest', aspect='equal', cmap='jet')
plt.xticks(range(0, nregions), conn.region_labels, fontsize=7, rotation=90)
plt.yticks(range(0, nregions), conn.region_labels, fontsize=7)
cb = plt.colorbar(shrink=0.2)
cb.set_label('Weights', fontsize=14)

# Tracts lengths
plt.subplot(122)
plt.imshow(conn.tract_lengths, interpolation='nearest', aspect='equal', cmap='jet')
plt.xticks(range(0, nregions), conn.region_labels, fontsize=7, rotation=90)
plt.yticks(range(0, nregions), conn.region_labels, fontsize=7)
cb = plt.colorbar(shrink=0.2)
cb.set_label('Tract lenghts', fontsize=14)

fig.tight_layout()

plt.show()

- ## <font size="5" face="arial" color="black"> Model</font>

<p><div style="text-align: justify"><font size="4.5" face="time roman">A set of differential equations describing the local neural dynamics. There are a number of predefined models available in TVB, as an example here we will use the <b>Generic 2-dimensional Oscillator</b> model:</font></div></p>

\begin{align}
\dot{V} &= d \, \tau (-f V^3 + e V^2 + g V + \alpha W + \gamma I), \\
                \dot{W} &= \dfrac{d}{\tau}\,\,(c V^2 + b V - \beta W + a).
\end{align}

<p><div style="text-align: justify"><font size="4.5" face="time roman">
    We can explore it using the <b>phase-plane</b> tool in TVB.
</font></div></p>

In [ ]:
# Create and launch the phase-plane tool
g2d = models.Generic2dOscillator()                 # The model to investigate
heunint = integrators.HeunDeterministic(dt=1.0)    # Integrator to use.
                                                   # dt is the integration step

ppi_fig = PhasePlaneInteractive(model=g2d, integrator=heunint)
ppi_fig.show()

# Try changing the default parameters to
#    1) a = 3   (limit cycle)
#    2) b = 1   (two stable and unstable fixed points)

<p><div style="text-align: justify"><font size="4.5" face="time roman">In the main central panel of the window, you can see the phase-plane for the model, including arrows representing the vector field and coloured lines representing any nullclines present in this plane. Clicking on the phase-plane will launch a sample trajectory originating from where you clicked. Below the phase-plane is a panel which will show time-series for all state variables for any sample trajectories you initiate. All around the edges are sliders for adjusting model parameters and adjusting what is displayed. The red vertical lines in sliders indicate the initial values.</font></div></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman"> 
    For our simulation we set it to the limit-cycle regime.
</font></div></p>

In [ ]:
# Initialize the model
g2d = models.Generic2dOscillator(a=np.array(1.7402))
g2d

- ## <font size="5" face="arial" color="black">Coupling function</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">It is a function that is used to join the local <b>Model</b> dynamics at distinct spatial locations over the connections described in <b>Connectivity</b>. Proper setting of the parameters for this function requires some knowledge of the properties of both the model being used and the structure through which it is connected. For our present purposes, we happen to know that for this configuration of parameters of TVB's <b>Generic2dOscillator</b> connected through TVB's default connectivity matrix, a linear function with a slope of 0.0075 is a reasonable thing to use.</font></div></p>

In [ ]:
# Initialise a Coupling function.
G = np.array(0.0075)
con_coupling = coupling.Scaling(a=G)

- ## <font size="5" face="arial" color="black">Integrator</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">Now that we have defined our structure and dynamics, we need to select an integration scheme. While TVB supports a number of schemes, for most purposes you should use either <b>HeunDeterministic</b> or <b>HeunStochastic</b></font></div></p> 

<p><div style="text-align: justify"><font size="4.5" face="time roman">Note that the most important thing here is to use a step size that is small enough for the integration to be numerically stable.</font></div></p>


In [ ]:
# Initialise an Integrator scheme.
dt = 0.1                          # Integration step [ms]

# We can use the deteministic integrator:
# heunint = integrators.HeunDeterministic(dt=dt)

# Or stochastic integrator:
nsigma = 0.001                    # Standard deviation of the noise
hiss = noise.Additive(nsig=np.array([nsigma, 0]))
heunint = integrators.HeunStochastic(dt=dt, noise=hiss)

- ## <font size="5" face="arial" color="black">Monitors</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">The last component we need to define are some Monitors. Although there are Monitors which apply a biophysical measurement process to the simulated neural activity, such as EEG, MEG, etc, here we will select two simple monitors just to show the idea:
<pre></pre>

<ul>
<li>the <b>Raw</b> monitor takes no arguments and simply returns all the simulated data -- note: as a general rule, this monitor shouldn't be used for anything but very short simulations as the amount of data returned can become prohibitively large,</li>
<li>the <b>TemporalAverage</b> monitor averages over a time window of length <i>period</i> returning one time point every <i>period</i> ms.</li></ul>
</font></div></p>

In [ ]:
# Initialise some Monitors with period in physical time.
mon_raw = monitors.Raw()
mon_tavg = monitors.TemporalAverage(period=1)    # 1000 Hz

#Bundle them
what_to_watch = (mon_raw, mon_tavg)

***

<h1 align="center"><font size="6"face="arial" color="black">Go! Simulate</font></h1>

<p><div style="text-align: justify"><font size="4.5" face="time roman">The last step is to bring all these components together into a <b>Simulator</b> object. We then need to run the configure method, which basically just acts to calculate information necessary for the simulation that draws on specific combinations of the components.</font></div></p>

In [ ]:
# Initialise the Simulator.
sim = simulator.Simulator(model=g2d,
                          connectivity=conn,
                          conduction_speed=np.float(conn.speed),
                          coupling=con_coupling,
                          integrator=heunint,
                          monitors=what_to_watch)
sim.configure()

<p><div style="text-align: justify"><font size="4.5" face="time roman">Now, we can run the simulation. All we need to do is iterate for some length, which we provide in <i>ms</i>, and collect the output:</p>

In [ ]:
# Perform the simulation.
tic = tm.time()

(raw_time, raw_data), (tavg_time, tavg_data) = sim.run(simulation_length=10000.)

'simulation required %0.3f seconds.' % (tm.time()-tic)

<p><div style="text-align: justify"><font size="4.5" face="time roman">
    For each monitor, the simulation returns the time points and the associated data array.</p>
<p>The data arrays have shapes 
    n<sub>timepoints</sub> x n<sub>variables of interest</sub> x n<sub>regions</sub> x n<sub>modes</sub>.
</p>

In [ ]:
raw_time.shape, raw_data.shape

In [ ]:
# Remove the dimensions with one element for easier indexing
raw = np.squeeze(np.array(raw_data))
tavg = np.squeeze(np.array(tavg_data))
raw.shape

***

<h1 align="center"><font size="6"face="arial" color="black">Visualize our simulation</font></h1>

<p><div style="text-align: justify"><font size="4.5" face="time roman">And finally, we can look at the results of our simulation in terms of time series.</font></div></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman">The data returned by the simulator is in the form of a list of arrays. For most subsequent purposes it is much easier to deal with the data if it exists as a single contiguous array. And so we will do that now:</font></div></p>

In [ ]:
# Normalize the time series for easier visualization
nraw = raw / (np.max(raw, 0) - np.min(raw, 0))
ntavg = tavg / (np.max(tavg, 0) - np.min(tavg, 0))

In [ ]:
# Plot the raw time series 
fig1 = plt.figure(figsize=(10,8))
plt.plot(raw_time[:], nraw[:, :10] + np.r_[:10])
plt.title('Raw Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(10), conn.region_labels, fontsize=10)

# Plot the temporally averaged time series
fig2 = plt.figure(figsize=(10,8))
plt.plot(tavg_time[:], ntavg[:, :10] + np.r_[:10])
plt.title('Temporally Averaged Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(10), conn.region_labels, fontsize=10)

plt.show()

<h1><font size="6"face="arial" color="black"> 2. Analyse our simulation</font></h1>

## <font size="5" face="arial" color="black"> 2.1 Functional Connectivity (FC)</font>

<p><div style="text-align: justify"><font size="4.5" face="time roman"><b>Functional Connectivity (FC)</b> describes the connectedness of two brain regions by means of the covariance between their time series. The classic and most widely used method to infer the strength of network interactions or functional connections consists in estimating the linear (Pearson) correlation coefficient between temporal signals. If two regions activate and deactivate at the same time, there is likely a functional connection. The Pearson correlation coefficient between two series $X$ and $Y$ of size $N$ is given by the following equation:</font></div></p> 

\begin{eqnarray}
    Corr(X, Y) = \dfrac{\sum_{n=1}^{N}(X_{n} - \bar{X})(Y_{n} - \bar{Y})}{\sqrt{\sum_{n=1}^{N}(X_{n} - \bar{X})^2}\sqrt{\sum_{n=1}^{N}(Y_{n} - \bar{Y})^2}}
\end{eqnarray}

<p><div style="text-align: justify"><font size="4.5" face="time roman">
    To calculate the FC, we ignore the first second of the simulation which might be affected by the initial conditions.
</font></div></p>

In [ ]:
fc = np.corrcoef(tavg[1000:].T)

<p><div style="text-align: justify"><font size="4.5" face="time roman">And now we display the resulting FC matrix:</font></div></p>

In [ ]:
# Visualize FC matrix
plt.figure(figsize=(8,8))
plt.imshow(fc, interpolation='nearest', cmap='jet')
plt.title('SIM FC', fontsize=20)
plt.xlabel('ROIs', fontsize=20); plt.ylabel('ROIs', fontsize=20)
plt.xticks([], fontsize=20); plt.yticks([], fontsize=20)
cb = plt.colorbar(shrink=0.7)
cb.set_label('PCC', fontsize=20)

plt.show()

- ## <font size="5" face="arial" color="black"> SC - FC comparison</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">We compare the empirical SC and simulated FC matrix by adopting as a measure of similarity between the two matrices the Pearson correlation between corresponding elements of the <b>upper (or lower)</b> triangular part of the matrices.</font></div></p>

In [ ]:
# Take upper triangular part of the matrices (excluding the self-connections).
sc_triu = sc[np.triu_indices(nregions, 1)]
fc_triu = fc[np.triu_indices(nregions, 1)]

# Compute Pearson correlation coefficients between SC and FC.
pcc = np.corrcoef(sc_triu, fc_triu)[0, 1]
pcc

## <font size="5" face="arial" color="black"> 2.2 Extract Resting-State Networks (RSNs)</font>

<p><div style="text-align: justify"><font size="4.5" face="time roman"> We will next study in more detail which features of the FC are explained by the model. For this, we will concentrate on seed-based correlation and principal components of the simulated FC.</font></div></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman">For the final analysis and plotting done here we will need few more tools: package scikit-learn (from conda) and file utils.py (from the repository; it should be placed in the same directory from which the notebook is run). </font></div></p>

In [ ]:
# ICA
from sklearn.decomposition import FastICA

# Brain maps
import utils

- ## <font size="5" face="arial" color="black">Seed-region correlation maps</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">A common visualization of FC specific to a given brain region is to pull out its row of the FC matrix and plot a colormap on the anatomy. We can do this with the simulated functional connectivity to identify which regions are highly coordinated with the seed region.</font></div></p>

In [ ]:
def plot_roi_corr_map(reg_name):
    roi = np.where(conn.ordered_labels == reg_name)[0]
    cs_m = fc
    rm = utils.cortex.region_mapping
    utils.multiview(cs_m[roi, rm], shaded=False, suptitle=reg_name, figsize=(10, 8))

<p><div style="text-align: justify"><font size="4.5" face="time roman">As a few examples of such maps, we take the medial parietal cortex as seed which take part of the well-known default-mode network (RSN1), as well as the middle prefrontal cortex for the dorsal attentional network (RSN2), the primary visual cortex for the visual network (RSN3), and the superior temporal cortex for the auditory network (RSN4). The seed regions use here are all in the right hemisphere.</font></div></p>

In [ ]:
for reg in 'rPCM rPFCM rV2 rTCS'.split():
    plot_roi_corr_map(reg)

- ## <font size="5" face="arial" color="black">Independent Component Analysis (ICA)</font></h3>

<p><div style="text-align: justify"><font size="4.5" face="time roman">Another common exploratory tool in resting-state data analysis, where the implicated regions or networks are not known a priori, is <b>Independent Component Analysis</b>, which extracts components with unique or independent statistical properties.</font></div></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman">For example, we can perform an ICA keeping 3 components from the above simulated data:</font></div></p>

In [ ]:
ica = FastICA(n_components=5, max_iter=1000, tol=0.01)
ica.fit(tavg[1000:, :])

<p><div style="text-align: justify"><font size="4.5" face="time roman">And then, visualize the different components:</font></div></p>

In [ ]:
for i, comp in enumerate(ica.components_[:3]):
    utils.multiview(comp[utils.cortex.region_mapping], shaded=False, 
                    suptitle='ICA %d' % (i, ), figsize=(10, 5))

<p><div style="text-align: justify"><font size="4.5" face="time roman">These components are not selected 'by hand', but represent independent subnetworks during the simulated resting state activity.</font></div></p>

<p><div style="text-align: justify"><font size="4.5" face="time roman">Finally, we point out, that commonly ICA analyses of fMRI are done at a group level to identify spatial patterns which are reproducible across subjects, whereas in this application to this simulation, spatial components may reflect as much non-robust, spontaneous fluctuations of the network passing through various state as the dominant rest state networks identified in human rest state.</font></div></p>

<h1><font size="6" face="arial" color="#609BC4">That is all folks -- so, what now?</font></h1>

***

<p><div style="text-align: justify"><font size="4.5" face="time roman">And that is it for this tutorial. These results are starting point, from which we can base our next simulations, going in directions such as:

<br></br>
<ul>
<li>using our own connectivity data in TVB's scripting interface,</li>
<li>perform a parameter sweep to identify regions of improved empirical and model FC correlations</li>
<li>etc.</li>
</ul>
</font></div></p>

<br></br>
<p><div style="text-align: justify"><font size="4.5" face="time roman">We hope this has been a useful tutorial and welcome any comments or questions.</font></div></p>